In [305]:
from __future__ import division
%matplotlib inline
from collections import defaultdict, OrderedDict
from gensim.models import Word2Vec, TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import identity
from util_kw import *
import math
import matplotlib.pyplot as plt
import os
import re
import numpy as np
from functools import partial

In [8]:
def get_words(journ, cw, dep, tn):
    path_models = 'C:\\Users\\dell\\Desktop\\Castes\\processed\\word2vec_models'
    wvm = Word2Vec.load(path_models+'\\'+journ+'_with_phraser').wv
    vocab = wvm.vocab
    all_words = [x for x in vocab.keys()]
    l_words_dict = {}
    for word in cw:
        if word in all_words:
            word_add_(word, wvm, l_words_dict, depth=dep, topn=tn)
        else:
            print ("Word "+word+" not found.")

    l_words_simpl = {}
    for w in l_words_dict :
        l_words_simpl[w] = 1.0
        for ww in l_words_dict[w] :
            l_words_simpl[ww[0]]=ww[1]
    return l_words_simpl

In [199]:
def get_corpus(myp_p, lista):
#    bigram = Phraser.load(path_phraser_models+journ+'_00_bigramphraser')
    pos_symbols = ['VB','VBD','VBG','VBN','VBP','VBP','VBZ','JJ','JJR','JJS']
    lines = []
    c = 0

    for fname in os.listdir(myp_p):
        for line in open(os.path.join(myp_p, fname)).readlines():
            c += 1
            for token in line.split():
                if token.split('\\')[0] in lista :
                    lines.append(line)                                 
    lines = [[token.split('\\')[0].lower() for token in line.split() if token.split('\\')[1] in pos_symbols ] 
             for line in lines]
    
    frequency = defaultdict(int)
    for line in lines:
        for token in line:
            frequency[token] += 1
    
    lines = [[token for token in line if frequency[token] > 4] for line in lines]

    print c, len(lines), len(lines)/c
    dic = corpora.Dictionary(lines)
    corp = [dic.doc2bow(line) for line in lines]
    return dic, corp
   

In [200]:
#list1 =get_words('IN-indianexpress-india',lower_caste_keywords+upper_caste_keywords,2,6)
#list2 =[]
#for w in list1.keys():
#    list2 = list2 + w.split('_')
#list2 = set(list2)
list2 = ['dalit','dalits']
dictionary, corpus = get_corpus('C:\\Users\\dell\\Desktop\\Castes\\processed_sentences\\IN-indianexpress-india\\', list2)


766077 11159 0.0145664208689


In [387]:
'''
Term frequency weighing:
    n - natural,
    l - logarithm,
    a - augmented,
    b - boolean,
    L - log average.
Document frequency weighting:
    n - none,
    t - idf,
    p - prob idf.
Document normalization:
    n - none,
    c - cosine.
'''

def updated_wlocal(tf, n_tf="b"):
    tf = tf / len(tf)
    if n_tf == "n":
        return tf
    elif n_tf == "l":
        return 1 + np.log(tf) / np.log(2)
    elif n_tf == "a":
        return 0.5 + (0.5 * tf / tf.max(axis=0))
    elif n_tf == "b":
        return tf.astype('bool').astype('int')
    elif n_tf == "L":
        return (1 + np.log(tf) / np.log(2)) / (1 + np.log(tf.mean(axis=0) / np.log(2)))
    
def updated_wglobal(docfreq, totaldocs, n_df="t"):
    if n_df == "n":
        return identity(docfreq)
    elif n_df == "t":
        return np.log(1.0 * totaldocs / docfreq) / np.log(2)
    elif n_df == "p":
        return np.log((1.0 * totaldocs - docfreq) / docfreq) / np.log(2)
    elif n_df == "a":
        return 10.0 + np.log((1.0 * totaldocs - docfreq) / docfreq) / np.log(2)
    
    
tfidf = TfidfModel(corpus, wlocal= updated_wlocal, wglobal=updated_wglobal, normalize=False)

words_n=tfidf[[(x,1) for x in range(0,len(dictionary))]]

words_tfidf={}
for x in words_n :
    words_tfidf[dictionary[x[0]]]=x[1]
    
for key, value in sorted(words_tfidf.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    if (value > 0.1) and (key not in [u'’',u'“',u'”',u'‘',u'\'s',u'—']) : print "%s : %s" % (key, value)

pathankot : 12.4459201272
zpsc : 11.8609576265
vigilant : 11.8609576265
vidarbha : 11.8609576265
truth : 11.8609576265
tooting : 11.8609576265
three-time : 11.8609576265
somalia : 11.8609576265
sinha-starrer : 11.8609576265
siddiqui : 11.8609576265
shyora : 11.8609576265
no-confidence : 11.8609576265
manifesto : 11.8609576265
indhira : 11.8609576265
halts : 11.8609576265
gowalikar : 11.8609576265
gbd : 11.8609576265
eliminating : 11.8609576265
disqualifying : 11.8609576265
brahmanical : 11.8609576265
adg : 11.8609576265
yashpal : 11.4459201272
x4 : 11.4459201272
utmost : 11.4459201272
urjit : 11.4459201272
uppal : 11.4459201272
ugc : 11.4459201272
tumour : 11.4459201272
tide : 11.4459201272
sucha : 11.4459201272
sleep : 11.4459201272
silver : 11.4459201272
searched : 11.4459201272
root : 11.4459201272
regressive : 11.4459201272
raid : 11.4459201272
puducherry : 11.4459201272
prospective : 11.4459201272
preside : 11.4459201272
premises : 11.4459201272
pet : 11.4459201272
patan : 11.4459

jaimat-e-ulema-e-hind : 10.8609576265
jaggi : 10.8609576265
ites : 10.8609576265
isl : 10.8609576265
iranian : 10.8609576265
ipltechnologyapple : 10.8609576265
introducing : 10.8609576265
intolerance : 10.8609576265
interfere : 10.8609576265
interactive : 10.8609576265
interacted : 10.8609576265
integral : 10.8609576265
insult : 10.8609576265
instated : 10.8609576265
instagramindialive : 10.8609576265
injured01:24 : 10.8609576265
ingreg : 10.8609576265
inentertainmentkunal : 10.8609576265
indycar : 10.8609576265
indulgent : 10.8609576265
indicts : 10.8609576265
indicted : 10.8609576265
indiapm : 10.8609576265
indiajnu : 10.8609576265
india07:28 : 10.8609576265
india01:36 : 10.8609576265
india-pakistan : 10.8609576265
inciting : 10.8609576265
incident.kaushik : 10.8609576265
inaugurating : 10.8609576265
imposes : 10.8609576265
imports02:21 : 10.8609576265
immune : 10.8609576265
immense : 10.8609576265
imitabletechnologyoneplus : 10.8609576265
iata : 10.8609576265
hoursat : 10.8609576265

etc : 9.86095762645
enhanced : 9.86095762645
emergessportsazhar : 9.86095762645
eden : 9.86095762645
earth-shattering : 9.86095762645
earned : 9.86095762645
du : 9.86095762645
drinking : 9.86095762645
dragentertainmentkabali : 9.86095762645
doaba : 9.86095762645
dismisses : 9.86095762645
dip : 9.86095762645
deleted : 9.86095762645
dared : 9.86095762645
creative : 9.86095762645
cr : 9.86095762645
covering : 9.86095762645
counterproductive : 9.86095762645
congress-ncp03:31 : 9.86095762645
complex : 9.86095762645
combat : 9.86095762645
closure : 9.86095762645
chupa : 9.86095762645
chopped : 9.86095762645
challenged : 9.86095762645
busy : 9.86095762645
broaden : 9.86095762645
briefly : 9.86095762645
breathing : 9.86095762645
blazing : 9.86095762645
bite : 9.86095762645
beware : 9.86095762645
babasaheb : 9.86095762645
attract : 9.86095762645
athletes : 9.86095762645
assault : 9.86095762645
ashamed : 9.86095762645
appnokia : 9.86095762645
appearing : 9.86095762645
appealing : 9.86095762645
a

jaitleyopinionas : 7.8020639374
inmust : 7.8020639374
handed : 7.8020639374
eternal : 7.8020639374
cleared : 7.8020639374
cause : 7.8020639374
built : 7.8020639374
believes : 7.8020639374
basic : 7.8020639374
barred : 7.8020639374
await : 7.8020639374
allotted : 7.8020639374
abram : 7.8020639374
1-2 : 7.8020639374
witnessed : 7.7734947852
white : 7.7734947852
waterentertainmentalia : 7.7734947852
videoentertainmenthappy : 7.7734947852
uniform : 7.7734947852
termed : 7.7734947852
sentenced : 7.7734947852
scst : 7.7734947852
sanjayentertainmentangry : 7.7734947852
replace : 7.7734947852
reaffirm : 7.7734947852
rajancitiesdelhi : 7.7734947852
q3 : 7.7734947852
permanent : 7.7734947852
ntr : 7.7734947852
newsthe : 7.7734947852
mega : 7.7734947852
mean : 7.7734947852
locked : 7.7734947852
interviewjun : 7.7734947852
intelligent : 7.7734947852
injurious : 7.7734947852
gathered : 7.7734947852
follow : 7.7734947852
expressindia'anti-social : 7.7734947852
entertainentertainmenthappy : 7.7734947